In [70]:
import pandas as pd
from pathlib import Path
import duckdb
import pickle

import matplotlib
from matplotlib import pyplot as plt
#%matplotlib inline

import seaborn as sns

In [47]:
data_dir=Path('/Users/vladimirmirosnikov/PycharmProjects/streamlit_HW/data')

In [48]:
d_frames=dict(zip([f.name[:-4] for f in data_dir.glob("*.csv")], [pd.read_csv(f) for f in data_dir.glob("*.csv")]))

In [45]:
d_frames['D_loan'].head(3)

,ID_LOAN,ID_CLIENT
0,1753790658,106804370
1,1753790659,106804371
2,1753790660,106804372


In [46]:
d_frames['D_clients'].head(3)

,ID,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,Московская область,1,0
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,Читинская область,0,0
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,Иркутская область,0,1


In [47]:
d_frames['D_last_credit'].head(3)

,CREDIT,TERM,FST_PAYMENT,ID_CLIENT
0,8000.0,6,8650.0,106804370
1,21650.0,6,4000.0,106804371
2,33126.0,12,4000.0,106804372


In [48]:
d_frames['D_pens'].head(3)

,ID,FLAG,COMMENT
0,1,0,пенсионер
1,2,1,не пенсионер


In [49]:
d_frames['D_salary'].head(3)

,FAMILY_INCOME,PERSONAL_INCOME,ID_CLIENT
0,от 20000 до 50000 руб.,20000.0,106809321
1,от 20000 до 50000 руб.,14000.0,106815561
2,от 10000 до 20000 руб.,15000.0,106811521


In [50]:
d_frames['D_target'].head(3)

,AGREEMENT_RK,ID_CLIENT,TARGET
0,59910150,106804370,0
1,59910230,106804371,0
2,59910525,106804372,0


In [51]:
d_frames['D_work'].head(3)

,ID,FLAG,COMMENT
0,1,0,работает
1,2,1,не работает
2,3,2,не известно


In [52]:
d_frames['D_job'].head(3)

,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,ID_CLIENT
0,Торговля,Рабочий,Вспомогательный техперсонал,18.0,106804370
1,Торговля,Рабочий,Участие в основ. деятельности,97.0,106804371
2,Информационные технологии,Специалист,Участие в основ. деятельности,84.0,106804372


In [53]:
d_frames['D_close_loan'].head(3)

,ID_LOAN,CLOSED_FL
0,1753790658,1
1,1753790659,1
2,1753790660,1


# "Покрутим" таблицы

In [57]:
d_frames['D_target'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   AGREEMENT_RK  15223 non-null  int64
 1   ID_CLIENT     15223 non-null  int64
 2   TARGET        15223 non-null  int64
dtypes: int64(3)
memory usage: 356.9 KB


In [59]:
d_frames['D_target'].AGREEMENT_RK.nunique(), d_frames['D_target'].ID_CLIENT.nunique()

(15223, 15223)

In [63]:
d_frames['D_clients'].columns

Index(['ID', 'AGE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS', 'CHILD_TOTAL',
       'DEPENDANTS', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
       'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE',
       'POSTAL_ADDRESS_PROVINCE', 'FL_PRESENCE_FL', 'OWN_AUTO'],
      dtype='object')

In [65]:
d_frames['D_clients'].ID.nunique(), len(d_frames['D_clients'])

(16000, 16000)

In [50]:
D_target=d_frames['D_target']
D_clients=d_frames['D_clients']
D_salary=d_frames['D_salary']
D_close_loan=d_frames['D_close_loan']
D_loan=d_frames['D_loan']
D_close_loan=d_frames['D_close_loan']
D_salary=d_frames['D_salary']

In [51]:
len(D_salary), D_salary.ID_CLIENT.nunique()

(15523, 15223)

In [80]:
len(D_close_loan), D_close_loan.ID_LOAN.nunique()

(21126, 21126)

In [83]:
len(D_loan), D_loan.ID_LOAN.nunique()

(21126, 21126)

In [85]:
len(D_close_loan), D_close_loan.ID_LOAN.nunique()

(21126, 21126)

In [87]:
D_close_loan.CLOSED_FL.value_counts()

CLOSED_FL
1    11446
0     9680
Name: count, dtype: int64

In [52]:
query="""
select
a.*
, b.AGE
, b.GENDER
, b.CHILD_TOTAL
, b.DEPENDANTS
, b.SOCSTATUS_WORK_FL
, b.SOCSTATUS_PENS_FL
, sal.PERSONAL_INCOME
, n_loans.LOAN_NUM_TOTAL
, n_loans.LOAN_NUM_CLOSED
from D_target as a 
left join D_clients as b 
    on a.ID_CLIENT=b.ID
left join D_salary as sal 
    on a.ID_CLIENT=sal.ID_CLIENT
left join 
    (select
    ln.ID_CLIENT
    , count(distinct ln.ID_LOAN) as LOAN_NUM_TOTAL
    , sum(cl_ln.CLOSED_FL) as LOAN_NUM_CLOSED
    from D_loan ln
    left join D_close_loan cl_ln on ln.ID_LOAN=cl_ln.ID_LOAN
    group by 1) n_loans on a.ID_CLIENT=n_loans.ID_CLIENT
"""

In [53]:
df_client_agg=duckdb.sql(query).df()

In [54]:
df_client_agg.head()

,AGREEMENT_RK,ID_CLIENT,TARGET,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,59910150,106804370,0,49,1,2,1,1,0,5000.0,1,1.0
1,59910525,106804372,0,52,1,4,0,1,0,9000.0,2,1.0
2,59910803,106804373,0,39,1,1,1,1,0,25000.0,1,1.0
3,59911781,106804374,0,30,0,0,0,1,0,12000.0,2,1.0
4,59911784,106804375,0,29,0,0,0,1,0,12000.0,2,1.0


In [55]:
df_client_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15523 entries, 0 to 15522
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGREEMENT_RK       15523 non-null  int64  
 1   ID_CLIENT          15523 non-null  int64  
 2   TARGET             15523 non-null  int64  
 3   AGE                15523 non-null  int64  
 4   GENDER             15523 non-null  int64  
 5   CHILD_TOTAL        15523 non-null  int64  
 6   DEPENDANTS         15523 non-null  int64  
 7   SOCSTATUS_WORK_FL  15523 non-null  int64  
 8   SOCSTATUS_PENS_FL  15523 non-null  int64  
 9   PERSONAL_INCOME    15523 non-null  float64
 10  LOAN_NUM_TOTAL     15523 non-null  int64  
 11  LOAN_NUM_CLOSED    15523 non-null  float64
dtypes: float64(2), int64(10)
memory usage: 1.4 MB


In [56]:
df_client_agg.iloc[:, 2:].describe()

,TARGET,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000,15523.000000
mean,0.119371,40.400438,0.653997,1.098886,0.644850,0.909360,0.134639,13848.041638,1.387296,0.751594
std,0.324235,11.607242,0.475709,0.996748,0.812663,0.287105,0.341348,8998.618992,0.794241,0.989253
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000
25%,0.000000,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8000.000000,1.000000,0.000000
50%,0.000000,39.000000,1.000000,1.000000,0.000000,1.000000,0.000000,12000.000000,1.000000,0.000000
75%,0.000000,50.000000,1.000000,2.000000,1.000000,1.000000,0.000000,17000.000000,2.000000,1.000000
max,1.000000,67.000000,1.000000,10.000000,7.000000,1.000000,1.000000,250000.000000,11.000000,11.000000


In [57]:
df_client_agg.dtypes

AGREEMENT_RK           int64
ID_CLIENT              int64
TARGET                 int64
AGE                    int64
GENDER                 int64
CHILD_TOTAL            int64
DEPENDANTS             int64
SOCSTATUS_WORK_FL      int64
SOCSTATUS_PENS_FL      int64
PERSONAL_INCOME      float64
LOAN_NUM_TOTAL         int64
LOAN_NUM_CLOSED      float64
dtype: object

In [14]:
df_client_agg.columns[2:]

Index(['TARGET', 'AGE', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS',
       'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'LOAN_NUM_TOTAL',
       'LOAN_NUM_CLOSED'],
      dtype='object')

In [61]:
2**32

4294967296

In [62]:
for col in df_client_agg.columns[2:]:
    df_client_agg[col]=df_client_agg[col].astype('int32')

In [63]:
df_client_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15523 entries, 0 to 15522
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   AGREEMENT_RK       15523 non-null  int64
 1   ID_CLIENT          15523 non-null  int64
 2   TARGET             15523 non-null  int32
 3   AGE                15523 non-null  int32
 4   GENDER             15523 non-null  int32
 5   CHILD_TOTAL        15523 non-null  int32
 6   DEPENDANTS         15523 non-null  int32
 7   SOCSTATUS_WORK_FL  15523 non-null  int32
 8   SOCSTATUS_PENS_FL  15523 non-null  int32
 9   PERSONAL_INCOME    15523 non-null  int32
 10  LOAN_NUM_TOTAL     15523 non-null  int32
 11  LOAN_NUM_CLOSED    15523 non-null  int32
dtypes: int32(10), int64(2)
memory usage: 849.0 KB


In [64]:
df_client_agg.to_csv('data/df_client_agg.csv', index=False)

In [2]:
df_client_agg=pd.read_csv('data/df_client_agg.csv')

In [65]:
df_client_agg.head()

,AGREEMENT_RK,ID_CLIENT,TARGET,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,59910150,106804370,0,49,1,2,1,1,0,5000,1,1
1,59910525,106804372,0,52,1,4,0,1,0,9000,2,1
2,59910803,106804373,0,39,1,1,1,1,0,25000,1,1
3,59911781,106804374,0,30,0,0,0,1,0,12000,2,1
4,59911784,106804375,0,29,0,0,0,1,0,12000,2,1


In [66]:
len(df_client_agg.columns[2:])

10

In [ ]:
help(

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(25, 20))



axes[0][0].hist(df_client_agg['AGE'])
axes[0][0].set_title('Distribution by AGE')

axes[0][1].hist(df_client_agg['GENDER'])
axes[0][1].set_title('Distribution by GENDER')

axes[1][0].hist(df_client_agg['CHILD_TOTAL'])
axes[1][0].set_title('Distribution by CHILD_TOTAL')

axes[1][1].hist(df_client_agg['DEPENDANTS'])
axes[1][1].set_title('Distribution by DEPENDANTS')

axes[2][0].hist(df_client_agg['SOCSTATUS_WORK_FL'])
axes[2][0].set_title('Distribution by SOCSTATUS_WORK_FL')

axes[2][1].hist(df_client_agg['SOCSTATUS_PENS_FL'])
axes[2][1].set_title('Distribution by SOCSTATUS_PENS_FL')

axes[3][0].hist(df_client_agg['PERSONAL_INCOME'])
axes[3][0].set_title('Distribution by PERSONAL_INCOME')

axes[3][1].hist(df_client_agg['LOAN_NUM_TOTAL'])
axes[3][1].set_title('Distribution by LOAN_NUM_TOTAL')

axes[4][0].hist(df_client_agg['LOAN_NUM_CLOSED'])
axes[4][0].set_title('Distribution by LOAN_NUM_CLOSED')

axes[4][1].hist(df_client_agg['TARGET'])
axes[4][1].set_title('Distribution by TARGET')


In [69]:
df_client_agg.iloc[:, 2:].corr()

,TARGET,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
TARGET,1.000000,-0.104542,-0.030849,0.010917,0.051368,0.078173,-0.084105,0.092851,-0.032437,-0.061603
AGE,-0.104542,1.000000,0.145340,0.209761,-0.285184,-0.454048,0.561547,-0.090645,0.023883,-0.000987
GENDER,-0.030849,0.145340,1.000000,0.050423,-0.081410,-0.104164,0.133761,-0.239444,0.008725,0.011885
CHILD_TOTAL,0.010917,0.209761,0.050423,1.000000,0.510148,-0.040943,0.059897,-0.053580,0.036906,0.006423
DEPENDANTS,0.051368,-0.285184,-0.081410,0.510148,1.000000,0.201100,-0.248439,0.035058,0.055113,0.042516
SOCSTATUS_WORK_FL,0.078173,-0.454048,-0.104164,-0.040943,0.201100,1.000000,-0.800395,0.198962,0.036425,0.042756
SOCSTATUS_PENS_FL,-0.084105,0.561547,0.133761,0.059897,-0.248439,-0.800395,1.000000,-0.141243,-0.008661,-0.020764
PERSONAL_INCOME,0.092851,-0.090645,-0.239444,-0.053580,0.035058,0.198962,-0.141243,1.000000,-0.036433,-0.025379
LOAN_NUM_TOTAL,-0.032437,0.023883,0.008725,0.036906,0.055113,0.036425,-0.008661,-0.036433,1.000000,0.859516
LOAN_NUM_CLOSED,-0.061603,-0.000987,0.011885,0.006423,0.042516,0.042756,-0.020764,-0.025379,0.859516,1.000000


In [26]:
df_client_agg.columns[3:]

Index(['AGE', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL',
       'SOCSTATUS_PENS_FL', 'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED'],
      dtype='object')

In [28]:
df_client_agg.iloc[:, 2:].describe()

,TARGET,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000
mean,0.119030,40.406096,0.654536,1.099389,0.645208,0.909610,0.134468,1.387769,0.751889
std,0.323835,11.601068,0.475535,0.995411,0.812252,0.286748,0.341165,0.793566,0.988499
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,0.000000,39.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,0.000000,50.000000,1.000000,2.000000,1.000000,1.000000,0.000000,2.000000,1.000000
max,1.000000,67.000000,1.000000,10.000000,7.000000,1.000000,1.000000,11.000000,11.000000
